In [ ]:
%matplotlib inline

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Background

This starts with a version of the Dipy Tracking Quickstart Tutorial (http://nipy.org/dipy/examples_built/tracking_quick_start.html#example-tracking-quick-start) for beginners to python. We then go through two more advanced methods for working with streamline data: Quickbundles (link) and Recobundles (link).

NOTE: For this tutorial, we assume you have eddy-corrected (https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/eddy) (and if you can susceptibility-corrected; https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/topup) your data

For more examples:
http://nipy.org/dipy/examples_built/

# Download and Load our data 
(if we don't have it already)

In [ ]:
from dipy.data import fetch_stanford_hardi, read_stanford_hardi
fetch_stanford_hardi()
img, gtab = read_stanford_hardi()

In [ ]:
data = img.get_data()

Let's check and see if our files match our expectations

In [ ]:
print(data.shape)

our data is 81 x 106 voxel resolution in-plane, 76 slices, and 160 volumes

In [ ]:
print(gtab.bvals)

... and we have our bvalues... 10 b0 volumes (no diffusion weighting) followed by 150 b2000 volumes

In [ ]:
print(gtab.bvecs.shape)

... and the shape of our bvecs make sense... 160 gradients by 3 dimenstions (xyz)

# Save a B0 image

Pick the first B0 image

In [ ]:
import numpy as np

index_of_first_B0 = np.where(gtab.b0s_mask)[0][0]
index_of_first_B0

Good... we saw from our list that the first volume is a b0 image so this matches our expectation

Now let's extract the first volume from our 4-D dataset

In [ ]:
b0_array = data[:,:,:,index_of_first_B0]

... and save it as a nifti

In [ ]:
from dipy.io.image import save_nifti

save_nifti('ohbm_dipy_b0.nii.gz', b0_array, img.affine)

# Do a quick brain segmentation

Note: the brain segmentation default parameters don't work, so we use specific ones here. On your own data, try the defaults first

In [ ]:
from dipy.segment.mask import median_otsu

maskdata, mask = median_otsu(data, 3, 2, False,
                             vol_idx=range(0, 50))

Now let's save the mask to our current directory

In [ ]:
save_nifti('ohbm_dipy_mask.nii.gz', mask*1, img.affine)

Now we will use nilearn plotting to check our mask

In [ ]:
import nilearn.plotting as nip

In [ ]:
nip.plot_roi('ohbm_dipy_mask.nii.gz', bg_img='ohbm_dipy_b0.nii.gz', 
             cmap=nip.cm.bwr_r, alpha=0.4)

# Fit a Tensor Model

In [ ]:
from dipy.reconst.dti import TensorModel

tensor_model = TensorModel(gtab, fit_method='WLS')
tensor_fit = tensor_model.fit(data, mask)

Now let's calculate fractional anisotropy (FA)

In [ ]:
fa = tensor_fit.fa

# clip FA values to between 0 and 1
fa = np.clip(fa, 0, 1)

# save the nifti file
save_nifti('ohbm_dipy_fa.nii.gz', fa, img.affine)

In [ ]:
nip.plot_anat('ohbm_dipy_fa.nii.gz')

Now let's visualize the tensor fit using the visualization tools

In [ ]:
from dipy.viz import actor, window
from dipy.reconst.dti import color_fa
from dipy.data import get_sphere

ren = window.Renderer()
rgb = color_fa(fa, tensor_fit.evecs)
    
evals = tensor_fit.evals[:,:,28:29]
evecs = tensor_fit.evecs[:, :, 28:29]
cfa = rgb[:, :, 28:29]
cfa /= cfa.max()

sphere = get_sphere('symmetric724') # we need a sphere for the render
ren.add(actor.tensor_slicer(evals, evecs, scalar_colors=cfa, 
                                sphere=sphere, scale=0.3))
# Save a snapshot as a png
window.record(ren, n_frames=1, out_path='ohbm_dipy_ren_ten.png', 
              size=(600, 600))

# Uncomment to show this interactively
#window.show(ren)

In [ ]:
# this will display snapshots inline so we can see our results
from IPython.display import Image

Image('ohbm_dipy_ren_ten.png')

# Fit a "higher order" more complex model

Examples of this are :
- Constrained Spherical Deconvolution (CSD) 
- Constant Solid Angle (CSA)/QBall Model.

First, we estimate a response function for CSD...

The response function is an estimation based on the data of the signal produced by a single coherent bundle of fibers. We will use voxels with very high FA to generate this from our data.

In [ ]:
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
from dipy.reconst.csdeconv import auto_response

response, ratio = auto_response(gtab, data, roi_radius=10, fa_thr=0.7)

csd_model = ConstrainedSphericalDeconvModel(gtab, response)

# Fit the data and calculate the peaks 

(we assume these are the dominant fiber directions)

In [ ]:
from dipy.direction import peaks_from_model

csd_peaks = peaks_from_model(model=csd_model,
                             data=data,
                             sphere=sphere,
                             mask=mask,
                             relative_peak_threshold=.5,
                             min_separation_angle=25,
                             parallel=True)

Check out our peaks on an axial slice by running this cell... a window will pop up 


In [ ]:
# Make the rendering with the peaks
window.clear(ren)
ren = window.Renderer()
ren.add(actor.peak_slicer(csd_peaks.peak_dirs,
                          csd_peaks.peak_values,colors=None))

# Uncomment this to interact with the render (a window will pop up)
#window.show(ren, size=(900, 900))

window.record(ren, out_path='ohbm_dipy_peaks.png', size=(900, 900))

In [ ]:
Image('ohbm_dipy_peaks.png')


In [ ]:
window.clear(ren)

Looking at the renders... what kinds of model limitations do we encounter with the tensor model?

# Tractography!!

Make a simple tissue classifier (gray/white/csf) using FA

In [ ]:
from dipy.tracking.local import ThresholdTissueClassifier

tissue_classifier = ThresholdTissueClassifier(fa, 0.15)

Generate seeds to track from in voxels with high FA... Let's seed all of the voxels with FA greater than 0.3 with one seed

In [ ]:
from dipy.tracking.utils import random_seeds_from_mask

seeds = random_seeds_from_mask(fa > 0.3, seeds_count=1)

In [ ]:
from dipy.tracking.local import LocalTracking
from dipy.tracking.streamline import Streamlines

streamline_generator = LocalTracking(csd_peaks, tissue_classifier,
                                     seeds, affine=np.eye(4),
                                     step_size=0.5)

streamlines = Streamlines(streamline_generator)

Now let's check how many streamlines we generated

In [ ]:
print(len(streamlines))

We can take a look at our streamlines now... let's write a function for this

In [ ]:
def show_sls(streamlines, out_path='ohbm_dipy_bigcluster.png', interact=False):    
    ren = window.Renderer()
    
    ren.add(actor.line(streamlines))

    if interact:
        window.show(ren, size=(900, 900))
    
    window.record(ren, out_path=out_path, size=(900, 900))
    return out_path

In [ ]:
Image(show_sls(streamlines))

Save our wholebrain tractography dataset as a .trk file

This streamline dataset contains many streamlines that we are not interested in... there is sometimes a shell of short streamlines around the periphery of the brain that we know are an artifact of our tracking procedure... in this case we just have a few where the mask fit was poor... we can remove them with a length threshold

In [ ]:
# Calculate the length of each streamline in the dataset
from dipy.tracking.utils import length

lengths = list(length(streamlines))

import matplotlib.pyplot as plt

fig_hist, ax = plt.subplots(1)
ax.hist(lengths, color='purple')
ax.set_xlabel('Length')
ax.set_ylabel('Count')

In [ ]:
# Set a length threshold
length_threshold = 20
long_streamlines = [] # initialize an empty list
for i,sl in enumerate(streamlines):
    if lengths[i] > length_threshold:
        long_streamlines.append(sl)

In [ ]:
Image(show_sls(long_streamlines))

Save our new track

In [ ]:
from nibabel.streamlines import save as save_trk
from nibabel.streamlines import Tractogram

save_trk(Tractogram(long_streamlines, affine_to_rasmm=img.affine),
         'ohbm_dipy_det_streamlines_long.trk')

# Explore our results with Quickbundles

In [ ]:
from dipy.segment.clustering import QuickBundles

In [ ]:
qb = QuickBundles(threshold=6.)
clusters = qb.cluster(streamlines)
print(len(clusters), "Clusters")

Let's take a look at the centroids...

In [ ]:
Image(show_sls(clusters.centroids))

In [ ]:
clsz = [len(cl) for cl in clusters]

fig_hist, ax = plt.subplots(1)
ax.hist(clsz, color='purple')
ax.set_xlabel('Length')
ax.set_ylabel('Count')

Let's take a look at our biggest cluster

In [ ]:
Image(show_sls(clusters[np.argmax(clsz)], interact=False))

# Recobundles

Let's try to automatically segment the CST using Recobundles... to do this we need to use the new nibabel streamline API 

In [ ]:
template_path_wb = '../OHBM_DIPY_TUTORIAL_DATA/kesh_template_Wholebrain.trk'
template_path_bundle = '../OHBM_DIPY_TUTORIAL_DATA/kesh_template_ArcuateL.trk'

In [ ]:
from dipy.align.streamlinear import whole_brain_slr, slr_with_qb, transform_streamlines

This is a function that will load a .trk file using the new nibabel streamline API

In [ ]:
def loadtgm_newapi(trkpath):
    trkloaded = nib.streamlines.trk.TrkFile.load(trkpath)
    hdrloaded = trkloaded.header
    tg=trkloaded.tractogram
    return tg

Let's load our bundle atlas (in this case it's the left Arcuate)

In [ ]:
import nibabel as nib

temp_atlasbundle_tg = loadtgm_newapi(template_path_bundle)
temp_atlasbundle_sls = temp_atlasbundle_tg.streamlines
temp_atlasbundle_as = nib.streamlines.array_sequence.ArraySequence(temp_atlasbundle_sls)

Now we'll make a function to display streamlines

In [ ]:
def genren(sls, sls2=None, niidata=None, roi1=None, roi2=None, rotx=-90, rotz=90, aff=None, putpath=None, interact=False, orient=True, colors=[(1,1,1)]):
    
    renderer = window.Renderer()
    
    if len(colors)>1:
        stream_actor = actor.line(sls, colors=colors[1])
    else:
        stream_actor=actor.line(sls)
    stream_actor.RotateX(rotx)
    stream_actor.RotateZ(rotz)
    renderer.add(stream_actor)
    
    if sls2 is not None:
        stream_actor2 = actor.line(sls2, colors=colors[0])
        stream_actor2.RotateX(rotx)
        stream_actor2.RotateZ(rotz)
        renderer.add(stream_actor2)
    
    if roi1 is not None and roi2 is not None:
        
        contour_actor1 = actor.contour_from_roi(roi1, affine=aff, color=(1., 1., 0.),
                                          opacity=0.5)
        contour_actor2 = actor.contour_from_roi(roi2, affine=aff, color=(1., 0., 0.),
                                          opacity=0.5)
        renderer.add(contour_actor1)
        renderer.add(contour_actor2)
        
    
    if niidata is not None:
        slice_actor = actor.slicer(niidata, affine=aff)
        renderer.add(slice_actor)

    #renderer.set_camera(position=(-176.42, 118.52, 128.20),
    #                    focal_point=(113.30, 128.31, 76.56),
    #                    view_up=(0.18, 0.00, 0.98))

    if orient:
        axes_actor = actor.axes(scale=(10, 10, 10), colorx=(1, 0, 0), colory=(0, 1, 0), colorz=(0, 0, 1), opacity=1)
        #Create an actor with the coordinate’s system axes where red = x, green = y, blue = z.
        renderer.add(axes_actor)

    if interact:
        window.show(renderer, size=(600, 600), reset_camera=False)
    if putpath is not None:
        window.record(renderer, out_path=putpath, size=(600, 600))
    del renderer
    return putpath

Display our bundle template

In [ ]:
Image(genren(temp_atlasbundle_sls, putpath='ohbm_dipy_adv_atlasbundle.png', interact=False))

Load and display our whole-brain template

In [ ]:
temp_wb_tg = loadtgm_newapi(template_path_wb)
temp_wb_sls = temp_wb_tg.streamlines
temp_wb_sls_as = nib.streamlines.array_sequence.ArraySequence(temp_wb_sls)

In [ ]:
Image(genren(temp_wb_sls, putpath='ohbm_dipy_adv_atlaswb.png'))

Load the wholebrain streamline dataset we generated earlier in the tutorial

In [ ]:
sub_wb_tg = loadtgm_newapi('ohbm_dipy_det_streamlines_long.trk')
sub_wb_sls = sub_wb_tg.streamlines
sub_wb_sls_as = nib.streamlines.array_sequence.ArraySequence(sub_wb_sls)

Make a function to roughly register the template-space wholebrain dataset and the subject-space wholebrain dataset we want to extract the Arcuate from

In [ ]:
def rough_reg(sub_fixed, temp_moving):
    #template moves to the subject space
    moved, transform, qb_centroids1, qb_centroids2 = whole_brain_slr(sub_fixed, temp_moving, 
                                                                     verbose=True,
                                                                     progressive=True)
    return moved, transform, qb_centroids1, qb_centroids2

In [ ]:
moved_temp2sub, xfm_temp2sub, qbc1_temp2sub, qbc2_temp2sub = rough_reg(sub_wb_sls_as, temp_wb_sls_as)

Before applying registration

In [ ]:
Image(genren(sub_wb_sls, temp_atlasbundle_sls, putpath='pre.png'))

obviously this is incorrect; we need to either
- apply the transform we calculated to the streamline template to bring the template into the patient's space
- apply the inverse transform to the patient's wholebrain streamlines to bring the patient into the template space)

Transform the Atlas bundle template into the subject space using our precomputed affine

In [ ]:
temp_atlasbundle_sls_subspace = temp_atlasbundle_tg.copy().apply_affine(xfm_temp2sub).streamlines

After applying registration we can see that the Arcuate bundle template is in the right space (subject-space)

In [ ]:
Image(genren(sub_wb_sls, temp_atlasbundle_sls_subspace, putpath='post.png'))

Here's a function to execute recobundles

In [ ]:
from dipy.segment.bundles import RecoBundles
from dipy.segment.clustering import qbx_and_merge

def run_rb(templatesls, bucketosls, cluster_map=None, pruning_thr=10):
    # try pruning thresh 10 if not specific drop to 5
    if cluster_map is None:
        cluster_map = qbx_and_merge(bucketosls, 
                                    thresholds=[40, 25, 20, 10])
    else:
        print("Loading provided cluster map")

    rb = RecoBundles(bucketosls, cluster_map=cluster_map, clust_thr=5)
    recognized_atlassp, rec_labels, recognized_ptsp = rb.recognize(model_bundle=templatesls,
                                                         model_clust_thr=5.,
                                                         reduction_thr=10, pruning_thr=pruning_thr)
    return recognized_ptsp, cluster_map

In [ ]:
rb_recognized_bundle, cluster_map = run_rb(temp_atlasbundle_sls_subspace, sub_wb_sls, pruning_thr=6)

In [ ]:
Image(genren(rb_recognized_bundle, putpath='recobundles.png', interact=False))

Now try playing with the pruning threshold in run_rb. What does this do?